In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import scipy.optimize
import sys
np.set_printoptions(edgeitems=10, linewidth=100000, threshold=sys.maxsize)
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [ ]:
# K input units
# N internal units
# L output units

# W_in   : N x K
# W      : N x N
# W_out  : L x (K + N + L)
# W_back : N x L

# n      : nth time step
# u(n)   : activation of input units (K-dim)
# x(n)   : activation of internal units (state) (N-dim)
# y(n)   : activation of output units
# f      : internal unit output function (also referred to as activation function), typically a sigmoid function like tanh

# internal activation is updated by
# x(n+1) = f(W_in u(n+1) + W x(n) + W_back y(n))
#           f([Nx1] + [Nx1] + [Nx1]) -> [Nx1]
# output computed by
# y(n+1) = f_out(W_out (u(n+1),x(n+1),y(n)))
#           f([Lx1]) -> []

# where (u(n+1),x(n+1),y(n)) is a concatenation

In [28]:
np.tanh(0.5)

0.46211715726000974

In [ ]:
class ESNModel():
    def __init__(self, inputUnits, inputConnGen, internalUnits, internalConnGen, outputUnits, outputConnGen, backConnGen, activation=np.tanh):
        self.inputUnits = inputUnits
        self.inputConnGen = inputConnGen
        self.internalUnits = internalUnits
        self.internalConnGen = internalConnGen
        self.outputUnits = outputUnits
        self.outputConnGen = outputConnGen
        self.backConnGen = backConnGen
    
    def makeInput(self):
        return self.inputConnGen(self.internalUnits, self.inputUnits)
        
    def makeInput(self):
        return self.inputConnGen(self.internalUnits, self.inputUnits)

In [5]:
np.random.uniform(0,1)

0.06193084452727837

In [31]:
f = lambda m,n : np.random.choice([0,1,1], (m,n), p=[0.95, 0.025, 0.025])

In [34]:
f(20,5)

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [6]:
np.random.random_integers(0,1,10)

/var/folders/27/3v164sd51z51b57cdr9dhxg40000gn/T/ipykernel_34402/3319591306.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  np.random.random_integers(0,1,10)


array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1])

In [12]:
-1*np.ones(10) + 2*np.random.randint(0,2,10)

array([ 1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.])

In [13]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [21]:
# A = np.random.choice([0,0.4,-0.4], (10,10), p=[0.95, 0.025, 0.025])
A = np.random.choice([0,1,1], (10,10), p=[0.95, 0.025, 0.025])
A

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [22]:
U,S,V = np.linalg.svd(A)
S

array([1.61803399, 1.        , 1.        , 1.        , 1.        , 1.        , 0.61803399, 0.        , 0.        , 0.        ])

In [26]:
AA = 0.4*A
AA

array([[0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0. , 0. ],
       [0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.4, 0. , 0. , 0. , 0. , 0. ]])

In [27]:
UU,SS,VV = np.linalg.svd(AA)
SS

array([0.6472136, 0.4      , 0.4      , 0.4      , 0.4      , 0.4      , 0.2472136, 0.       , 0.       , 0.       ])

In [20]:
float(7.72740661e-01)

0.772740661